<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" style="padding-right:10px;width:140px;float:left"></td>
<h2 style="white-space: nowrap">Neural Signals and Signal Processing (NX-421)</h2>
<hr style="clear:both"></hr>

Welcome to the laboratory computers for the course "Neural signals and signal processing". 
This week, we finish up the preprocessing of fMRI on some advanced points, which you might need when working on your mini-project.
We will then look at functional Near Infrared Spectroscopy (fNRIS).

In [1]:
%gui wx
import sys
import os

#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist, interactive_MCQ,get_json_from_file

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################
os.environ["DIPY_HOME"] = "/home/jovyan/Data"


#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri

import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

import ants
import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json

from IPython.display import display, HTML


In [2]:

# Inject custom CSS
custom_css ="""
<style>

  .fit {
    object-fit: cover;
  }

  .container {
    display: flex;
    align-items: center; /* Align blocks vertically in the middle */
    justify-content: flex-start; /* Align blocks to the left */
  }

  .imageDiv {
      background: #fff;
      display: block;height: 150px;width: 150px;padding: 10px;border-radius: 2px;box-shadow: 0 1px 4px rgba(0, 0, 0, 0.3), 0 0 40px rgba(0, 0, 0, 0.1) inset;flex-shrink: 0;
  }
    .arrow-right {
      width: 0; 
      height: 0; 
      border-top: 1em solid transparent;
      border-bottom: 1em solid transparent;
      border-left: 1em solid #000;
    }
      .col-md-10 {
        display: flex;
        align-items: center;
      }
    
    .bby {
        display: flex;
        justify-content: center;
        align-items: center;
        height: 100vh;
        font-family: Arial, sans-serif;
    }
    
    .flow-container {
        display: flex;
        align-items: center;
        justify-content:center;
    }
    
    .box-text-container {
        flex-direction: column;
        display: center;
        align-items: center;
        justify-content:center;
    }
    
    .step-box {
        background-color: lightgray;
        padding: 0.5em;
        margin: 0 0.5em;
        text-align: center;
        font-weight: bold;
        border-radius: 0.25em;
        border: 0.15em solid black;
        min-width: 6em;
    }
    
    .arrow {
        font-size: 1.5em;
        color: blue;
        font-weight: bold;
        margin: 0 0 0.5em;
    }
    
    .text {
        font-weight: bold;
        font-size: 0.9em;
        margin: 0 0.5em;
    }
    
    .box-wrapper {
        display: flex;
        align-items: center;
        padding: 0.5em;
        border: 0.15em solid black;
        border-radius: 10px;
    }
    
    /* Style for the text below the box */
    .kapt_2 {
        margin-top: 0.5em;
        font-size: 1em;
        font-weight: bold;
    }

    .disp_p {
        font-size:2.5em;
    }
"""

display(HTML(custom_css))

In [3]:
################
# Start FSLeyes (very neat tool to visualize MRI data of all sorts) within Python
################
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()

Gtk-Message: 21:29:09.313: Failed to load module "canberra-gtk-module"

(ipykernel_launcher.py:14302): Gtk-CRITICAL **: 21:29:09.521: gtk_window_resize: assertion 'height > 0' failed
21:29:10: Debug: ScreenToClient cannot work when toplevel window is not shown
21:29:10: Debug: ScreenToClient cannot work when toplevel window is not shown
21:29:10: Debug: ScreenToClient cannot work when toplevel window is not shown
21:29:10: Debug: ScreenToClient cannot work when toplevel window is not shown
21:29:10: Debug: ScreenToClient cannot work when toplevel window is not shown
21:29:10: Debug: ScreenToClient cannot work when toplevel window is not shown
21:29:10: Debug: ScreenToClient cannot work when toplevel window is not shown
21:29:10: Debug: ScreenToClient cannot work when toplevel window is not shown
21:29:10: Debug: ScreenToClient cannot work when toplevel window is not shown
21:29:10: Debug: ScreenToClient cannot work when toplevel window is not shown
21:29:10: Debug: ScreenToClient canno

# Part 1: Advanced preprocessing (Optional)

This part is a continuation of last week. It is optional, but some parts (such as "Applying the transformation to all volumes") might be helpful for your miniprojects.

## 1.1 Field map unwarping

The field itself is not homogeneous, as you've seen in class. This means, in turn, that there are distortions in the acquisition.
We can try to correct for it, through field maps, provided they've been acquired.

### What are field maps ? 

Field maps are maps of the magnetic field (hence their name). They are acquired during an experimental session to capture parts where the MRI's magnetic field might present inhomogeneities. These inhomogeneities will, in turn, cause distortions in the signal which are not part of the subject's anatomy, as well as signal drop (places where the contrast becomes very small between tissues). Such artefacts should obviously be removed.
This is where fieldmaps are typically coming into play: by knowing how your scanner is distorting your signal, you can hope to correct for it - to some amount.

The first step is - naturally - to acquire fieldmaps.

Fortunately this is the case in our dataset - but you will need to download them as we have avoided loading them for you - on purpose!

To make sure you've understood how to load datasets, here is the dataset of interest: https://openneuro.org/datasets/ds004226/versions/1.0.0

<img src="imgs/dataset_screen.png">

Your first task is to load:
- Subject 001 data files, including the fieldmap files, located in the fmap subfolder (WITH the JSON sidecars!)

Out of convenience, we already provide you with the openneuro-py case. Modify the command-line run below to include *all files in the fmap subfolder of sub-01*.

In [ ]:
import subprocess

dataset_id = 'ds004226'
subject = '001'

sample_path = "/home/jovyan/Data/dataset"
mkdir_no_exist(sample_path)
bids_root = op.join(os.path.abspath(""),sample_path, dataset_id)
deriv_root = op.join(bids_root, 'derivatives')
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')

fmap_path = op.join(bids_root, 'sub-001', 'fmap')
subject_dir = 'sub-{}'.format(subject)

##################
# Solution
# There are two solutions
# The easiest is simply to include all data of subject-01
# The other is to add one line for the fieldmaps
##################
# Change the command below to include files in the fmap subdirectory
# You should STILL be loading the EPI and anatomical
subprocess.run(["openneuro-py", "download", "--dataset", dataset_id, # Openneuro has for each dataset a unique identifier
                "--target-dir", bids_root,  # The path where we want to save our data. You should save your data under /home/jovyan/Data/[your dataset ID] to be 100% fool-proof
                "--include", op.join(subject_dir, 'anat','*'),# We are asking to get all files within the subject_dir/anat folder by using the wildcard *
                "--include", op.join(subject_dir, 'func','*'),# We are asking to get all files within the subject_dir/func folder by using the wildcard *
                "--include", op.join(subject_dir, 'fmap','*'),# We are asking to get all files within the subject_dir/fmap folder by using the wildcard *
               ], check=True)

# Simple variant to include everything from subject-01
subprocess.run(["openneuro-py", "download", "--dataset", dataset_id, # Openneuro has for each dataset a unique identifier
                "--target-dir", bids_root,  # The path where we want to save our data. You should save your data under /home/jovyan/Data/[your dataset ID] to be 100% fool-proof
                "--include", subject_dir # Effectively get all data
               ], check=True)

Again, remember this download might not be finished immediately :)
Now, assuming it *is*, let's have a look at what we have:

In [ ]:
print_dir_tree(bids_root, max_depth=5)

Look in the fmap folder. There are four files, corresponding to two fieldmap acquisitions. One is PA, the other is AP.

We need some parameters to be able to exploit these files.
In particular, we need to figure out:
- The phase encoding direction
- The total readout time

Your task is to figure out which keys to exploit for this.
Have a look at the code below (and feel free to play around a bit of course!) to setup the values properly.
To help you, we've loaded one of the two JSON sidecars:

In [ ]:
get_json_from_file(op.join(fmap_path, 'sub-001_acq-task_dir-{}_epi.json'.format('AP')))

In [ ]:
mkdir_no_exist(deriv_root)
mkdir_no_exist(preproc_root)
mkdir_no_exist(op.join(preproc_root, 'sub-001'))
mkdir_no_exist(op.join(preproc_root, 'sub-001', 'func'))
mkdir_no_exist(op.join(preproc_root, 'sub-001', 'anat'))
mkdir_no_exist(op.join(preproc_root, 'sub-001', 'fmap'))

In [ ]:
preproc_fmap_path = op.join(preproc_root, 'sub-001', 'fmap')
mkdir_no_exist(preproc_fmap_path)
direction_file = op.join(preproc_fmap_path, 'datain.txt')

f = open(direction_file, 'w')

for name in ['AP', 'PA']:
    data = get_json_from_file(op.join(fmap_path, 'sub-001_acq-task_dir-{}_epi.json'.format(name)))
    phase_dir = data['PhaseEncodingDirection'] # Extract here the phase encoding direction !
    total_readout_time = data['TotalReadoutTime'] # Extract here the total readout time !
    
    # We expect a specific format, namely x y z total_readout_time, where x,y and z are set to 1/-1 if and only if they are the phase
    # encoding direction, 0 otherwise.
    phase = [0, 0, 0, total_readout_time]
    is_neg = len(phase_dir) == 2 and phase_dir[1] == '-'
    phase_dir = phase_dir[0]
    phase[ord(phase_dir)-ord('i')] = -1 if is_neg else 1
    for i in range(3):
        f.write('{} {} {} {}\n'.format(phase[0], phase[1], phase[2], phase[3]))
f.close()

### 1.1.1 Creating the field map
Now, we will create the field map.
This process is tedious, sometimes hard to get right. First, let's look at the two fieldmaps.

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(op.join(fmap_path, 'sub-001_acq-task_dir-AP_epi.nii.gz'))
fsleyesDisplay.load(op.join(fmap_path, 'sub-001_acq-task_dir-PA_epi.nii.gz'))

As you notice, they are quite different with respect to their distortions. This is because they used different phase encoding directions (Anterior -> Posterior and Posterior -> Anterior, hence AP and PA).

Looking from these two encoding directions, we (or rather clever algorithm: <a href="https://web.mit.edu/fsl_v5.0.10/fsl/doc/wiki/topup.html">topup</a>) can build a complete map of the distortions on our EPI / fMRI file.

Here is the signature of topup:

In [ ]:
os.system("topup")

Not very clear, is it? You need to, in fact, conduct several steps. 

- First, AP and PA fieldmaps should be made a single file to be fed to topup.
- You also need to feed it the direction file we created above (which specifies encoding direction and readout time of our EPI).
- From the computed field, we need to convert it to radians, and finally, we obtain both a phase information for the field - in radians - and a magnitude information.

You are now ready to apply the fieldmap to correct distortions.

The function below conducts all these steps for you:

In [ ]:
def generate_fmap_AP_PA(direction_file):
    """
    From an AP/PA pair of files, generate the corresponding fieldmap files.

    Parameters
    ----------
    direction_file: str
        Path to a direction file (typically called datain.txt) indicating phase encoding direction, total 
        readout time and other relevant parameters for fmap computation
    """
    merged_phase_imgs = op.join(preproc_fmap_path, 'sub-001_acq-task_dir-fmap_merged')
    
    # Step 1: Combine AP and PA as single file
    subprocess.run(['fslmerge', '-t', merged_phase_imgs, 
                    op.join(fmap_path, 'sub-001_acq-task_dir-AP_epi.nii.gz'), 
                    op.join(fmap_path, 'sub-001_acq-task_dir-PA_epi.nii.gz')])
    
    # Step 2: Compute the fieldmap deformation with topup 
    # In this particular step, we feed in the direction file (ie the phase encoding direction, phase order etc, which we've saved above as direction file
    output_fmap = op.join(preproc_fmap_path, 'fieldmap_ex')
    unwarped_img = op.join(preproc_fmap_path, 'se_epi_unwarped')

    subprocess.run(['topup', 
                    '--imain={}'.format(merged_phase_imgs), 
                    '--datain={}'.format(direction_file),
                   '--config={}'.format('b02b0.cnf'),
                   '--fout={}'.format(output_fmap),
                   '--iout={}'.format(unwarped_img),
                   '-v'])
    
    # Step 3: Convert fmap units to rads
    subprocess.run(['fslmaths', output_fmap, '-mul', str(6.28), output_fmap + '_rads'])
    #fslmaths(output_fmap).mul(6.28).run(output_fmap + '_rads')
    
    # Step 4: Create magnitude fmap
    subprocess.run(['fslmaths', unwarped_img, '-Tmean', output_fmap + '_mag'])
    #fslmaths(unwarped_img).Tmean().run(output_fmap + '_mag')
    
    # Extract fmap brain using bet
    subprocess.run(['bet', output_fmap + '_mag',output_fmap + '_mag_brain', '-m', '-R'])
generate_fmap_AP_PA(direction_file)

It now remains to apply the fieldmap! 

To do so we will apply to the **first volume of our series to show you the result of distortion correction**.
Your first task is thus to extract the first volume by modifying the below command:

In [ ]:
file_to_trim = op.join(bids_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold.nii.gz')
mkdir_no_exist(op.join(preproc_root, 'sub-001', 'func'))
extracted_epi = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_vol_1')

###########
# Solution
# Extracting the first volume means we keep a single volume. 
# This is exactly as we did last week for the reference EPI used in epi_reg,
# but we take the volume number 0 instead of the middle one.
###########

# Select only the FIRST volume!
start_vol = 0 # Where should we start? (First volume is 0, not 1 !)
number_of_volumes = 1 # How many volumes should we keep?

fslroi(file_to_trim, extracted_epi, str(start_vol), str(number_of_volumes))

In [ ]:
fsleyesDisplay.load(extracted_epi)

Great! 
We are finally able to apply our fieldmap to the EPI. We can do so, using FUGUE!

In [ ]:
fmap_path_rad = op.join(preproc_root, 'sub-001', 'fmap', 'fieldmap_ex_rads')
epi_result= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_vol_1_fmap')

unwarpdir='y-'
dwell_time= data['EffectiveEchoSpacing'] # Although FUGUE wants the dwell time, the reported dwell time in the header is wrong: it should be in order of milliseconds (0.3 to 60ms is reasonable), but here it is on the order of microseconds. We use effective echo spacing instead, which is the same thing...And in correct order of magnitude.

subprocess.run(['fugue', '-i', extracted_epi, # Fieldmaps are applied to EPI
                '--loadfmap={}'.format(fmap_path_rad), # We used the fmap we just created
               '--dwell={}'.format(dwell_time), # The dwell time is necessary as input parameter
               '--unwarpdir={}'.format(unwarpdir), # The unwarp direction
               '-u', epi_result])


Let's visualize!

In [ ]:
fsleyesDisplay.load(epi_result)

You should observe the following two volumes:

<center>
<div style="display:inline-block;">
    <img src="imgs/uncorrected_brain.png" style="height: 200px;width:auto;border: blue 6px groove;" />
    <p style="text-align:center;">Before unwarp (sagittal)</p>
</div>
<div style="display:inline-block;">
    <img class="middle-img" src="imgs/corrected_brain.png"/ style="height: 200px;width:auto;border: green 6px groove;" />
    <p style="text-align:center;">After unwarp (sagittal)</p>
</div>
<br><br>
<div style="display:inline-block;">
    <img src="imgs/uncorrected_brain_2.png" style="height: 270px;width:250px;border: blue 6px groove;"/>
    <p style="text-align:center;">Before unwarp (axial)</p>
</div>
<div style="display:inline-block;">
    <img src="imgs/corrected_brain_2.png" style="height: 270px;width:250px;border: green 6px groove;" />
    <p style="text-align:center;">After unwarp (axial)</p>
</div>
</center>


In [ ]:
interactive_MCQ(4,1)

Do you think the fieldmap made an improvement? To drive your answer, feel free to inspect both volumes in FSLeyes. Observe the frontal and ventral regions. Do you notice anything different? Which one seems to match better what you'd expect from the brain anatomy ? 

<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b> Assessing quality of functional data 💡</b></p>
<p style='text-indent: 10px;'>
    When in doubt about what you observe, don't be afraid to go have a look at your T1 to compare against. If a structure in the functional shows up in the T1 but distorted, you'll find out faster this way.</p>
</span>
</div>

<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b>💡 Pay attention! 💡</b></p>
<p style='text-indent: 10px;'>
    Easy? Well, not always. Field maps for this dataset came in a specific format. But they can come in <b>many</b> different ways, meaning you will need to be very careful when recovering them. The steps outlined above in particular are only applicable in the case of having an AP-PA acquisition. Here is the full resource of FSL's FUGUE on field map unwarping: <a href=https://fsl.fmrib.ox.ac.uk/fsl/docs/#/registration/fugue>https://fsl.fmrib.ox.ac.uk/fsl/docs/#/registration/fugue</a> . Don't be afraid to refer to it, should you have a different format in a project!</p>
</span>
</div>


## 1.2 Combining transforms

We know how to perform motion-correction, and how to coregister an image to another. That's great!
But each transformation usually implies a step of interpolation (because the image is transformed and must be resampled). This interpolation means the resulting data is "corrupted" slightly. We would like to minimize the amount of interpolations to only once if possible.

Here is a reminder of the different spaces we've considered, as well as the different operations and how they move between spaces.

<center><img src="imgs/epi_transform_steps.png"></center>

In [ ]:
interactive_MCQ(4,2)

The order of transformations we would like to have is:

<body>
    <div class="flow-container">
        <div class="text">EPI</div>
        <div class="arrow">→</div>
        <div class="step-box">Motion correction<br>Mcflirt</div>
        <div class="arrow">→</div>
        <div class="text">EPI<br> (motion corrected)</div>
        <div class="arrow">→</div>
        <div class="step-box">Field unwarping + affine coregistration<br>epi_reg with fieldmap</div>
        <div class="arrow">→</div>
        <div class="text">EPI<br> (Anatomical space)</div>
        <div class="arrow">→</div>
        <div class="step-box">Normalization<br>Flirt or ANTs</div>
        <div class="arrow">→</div>
        <div class="text">EPI<br> (template space)</div>
    </div>

</body>

### 1.2.1 Combining fieldmap unwarping and EPI registration

FSL provides a way to compute the EPI to anatomical while combining it with the fieldmap unwarping. We will show you in this part how to do it.

But before applying all transforms, let's worry about doing the required preprocessing, namely:
- Motion correction
- Field unwarping
- EPI to anatomical coregistration
- Anatomical to template coregistration (Normalization): we will use the MNI152 1mm template that you know from the previous weeks


First, remark that motion correction is done by selecting a reference volume in the EPI to which all others are coregistered. By default, the middle EPI was used. 

Because we used in our fieldmap computation the first EPI, we need to use this one instead.

**It is critical that you pay attention to which image was used to compute your transformations, otherwise combining them won't make sense!**.

For this reason, let's now go over the entire pipeline and transformation steps, sticking to the first EPI. We extract it again with fslroi, and we re-run the motion correction.

In [ ]:
original_epi = op.join(bids_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold')
reference_epi = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_first-vol')
fslroi(original_epi, reference_epi, str(0), str(1))

Now, let's do motion correction. Recall that it is done on the **entire** EPI timeseries with mcflirt. We will explicitly give the first epi as reference this time around, to force FSL to use this volume and realign everyone to it!

In [ ]:
from fsl.wrappers import mcflirt

path_moco_data = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_moco')
mcflirt(infile=original_epi,o=path_moco_data, plots=True, report=True, dof=6, mats=True, reffile=reference_epi)

Fantastic! Now, because **the reference volume did not move at all** (since it is the reference to which everyone is realigned), we can use this volume as starting point to compute our other transforms: we're only missing the coregistration with fieldmap unwarping, as the normalization is obtained through the anatomical data :)



In [ ]:
subject_id = '001'
subject='sub-001'

# Relevant paths for anatomical preprocessing
anatomical_path = op.join(bids_root, subject, 'anat', 'sub-{}_T1w.nii.gz'.format(subject_id))
betted_brain_path = op.join(preproc_root, subject, 'anat', 'sub-{}_T1w'.format(subject_id))
segmentation_path = op.join(preproc_root, 'sub-001', 'anat', 'sub-001_T1w_fast')

mni_template = op.expandvars(op.join('$FSLDIR', 'data', 'standard', 'MNI152_T1_1mm_brain'))
anat_result = op.join(preproc_root, subject, 'anat', 'sub-{}_T1w_mni'.format(subject_id))
anat_2_mni_trans = op.join(preproc_root, subject, 'anat', 'sub-{}_T1w_2_mni_lin.mat'.format(subject_id))

# Relevant variables for epi_reg
output_path = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_anat-space')
dwell_time = 0.000620007
unwarpdir='y-'

##############
# Put here all steps which you might need to conduct before you can do epi_reg
# Hint: consider the anatomical preprocessing steps, look at week 2 if you forgot!
# For flirt, have a look at the omat argument to save the transform
##############

##############
# Solution
# We need to do the brain extraction, using bet as well as the segmentation, using fast, lastly we perform normalization to MNI152 space
##############
from fsl.wrappers import fast

subprocess.run(['bet', anatomical_path, betted_brain_path, '-m', '-R'])

fast(imgs=[betted_brain_path], out=segmentation_path, n_classes=3)

flirt(betted_brain_path, mni_template, out=anat_result, omat = anat_2_mni_trans)

#############
# Launching epi_reg with fieldmap unwarping.
# Careful to do it ON THE FIELDMAP CORRECTED VOLUME
# Note epi_reg will take a few minutes to compute the transform - feel free to bombard us with questions (or candy)
#############
reference_epi = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_first-vol')
fslroi(original_epi, reference_epi, str(0), str(1))
subprocess.run(['epi_reg','--epi={}'.format(reference_epi), 
                '--t1={}'.format(anatomical_path), 
                '--t1brain={}'.format(betted_brain_path), 
                '--out={}'.format(output_path),
                '--fmap={}'.format(op.join(preproc_root, 'sub-001', 'fmap', 'fieldmap_ex_rads')),
                '--fmapmagbrain={}'.format(op.join(preproc_root, 'sub-001', 'fmap', 'fieldmap_ex_mag_brain')),
                '--fmapmag={}'.format(op.join(preproc_root, 'sub-001', 'fmap', 'fieldmap_ex_mag')),
                '--wmseg={}'.format(op.join(preproc_root, 'sub-001', 'anat', 'sub-001_T1w_fast_pve_2')),
                '--echospacing={}'.format(dwell_time),
                '--pedir={}'.format(unwarpdir)])

print("Done with EPI to anatomical registration with fieldmap unwarping")

Beautiful! 

➡️ Inspect the two resulting files to ensure that nothing went wrong. In other words:
- Check that the MCFLIRT result is okay with respect to motion
- Check that the realignment following epi_reg made the EPI well aligned with the anatomical

In [ ]:
# Inspect here with fsleyes, such as fsleyesDisplay.load(yourEpi) :)

### 1.2.2 Getting the saved transformation

Applying the transformation to a single volume is nice, but we should still need to know where the transformation was saved, to apply it to all other volumes of interest.

Let's inspect our resulting folders:

In [ ]:
print_dir_tree(bids_root,max_depth=5)

The file <b>sub-001_task-sitrep_run-01_bold_anat-space_warp</b> corresponds to the <u>transformation</u> from EPI to anatomical file with the fieldmap unwarping applied.

In [ ]:
interactive_MCQ(4,3)

### 1.2.3 Combining the transforms

Let's list our available transforms as is:

 <table>
  <tr>
    <th>Transform</th>
    <th>Step</th>
    <th>File(s)</th>
    <th>Linear?</th>
  </tr>
  <tr>
    <td>Each EPI to reference EPI</td>
    <td>Motion correction (mcflirt) </td>
    <td>matrices in func/sub-001_task-sitrep_run-01_bold_moco.mat/</td>
    <td>Yes</td>
  </tr>
  <tr>
    <td>EPI to anat (with fieldmap)</td>
    <td>Functional to anat coregistration (epi_reg) </td>
    <td>func/sub-001_task-sitrep_run-01_bold_anat-space_warp.nii.gz</td>
    <td>No</td>
  </tr>
  <tr>
    <td>Anat to template</td>
    <td>Normalization (FLIRT or FNIRT or ANTs)</td>
    <td>anat/.mat</td>
    <td>Yes if FLIRT or ANTs with linear transform, no otherwise</td>
  </tr>
</table> 

We will now combine all these transformations, so that we apply one transformation and exactly one interpolation at the end to minimize errors.

We provide you with a function to do so just below, along with a function to apply the resulting combined transformation. Beware, all these functions operate on single volumes, not on 4D data!


In [ ]:
def combine_all_transforms(reference_volume, warp_save_name,  is_linear, epi_2_moco=None, epi_2_anat_warp=None, anat_2_standard_warp=None):
    """
    Combines transformation BEFORE motion correction all the way to standard space transformation
    The various transformation steps are optional. As such, the final warp to compute is based on 
    which transforms are provided.

    Parameters
    ----------
    reference_volume: str
        Reference volume. The end volume after all transformations have been applied, relevant for final resolution and field of view.
    warp_save_name: str
        Under which name to save the total warp
    is_linear: bool
        Whether the transformation is linear or non linear.
    epi_2_moco: str
        Transformation of the EPI volume to motion-correct it (located in the .mat/ folder of the EPI
    epi_2_anat_warp: str
        Transformation of the EPI volume to the anatomical space, typically obtained by epi_reg. Assumed to include fieldmap correction and thus be non-linear.
    anat_2_standard_warp: str
        Transformation of the anatomical volume to standard space, such as the MNI152 space. Might be linear or non linear, which affects is_linear value accordingly.
    """
    from fsl.wrappers import convertwarp
    args_base = {'premat': epi_2_moco, 'warp1': epi_2_anat_warp}
    if is_linear:
        args_base['postmat'] = anat_2_standard_warp
    else:
        args_base['warp2'] = anat_2_standard_warp
    args_filtered = {k: v for k, v in args_base.items() if v is not None}

    convertwarp(warp_save_name, reference_volume, **args_filtered)
    print("Done with warp conversion")

def apply_transform(reference_volume, target_volume, output_name, transform):
    """
    Applies a warp field to a target volume and resamples to the space of the reference volume.

    Parameters
    ----------
    reference_volume: str
        The reference volume for the final interpolation, resampling and POV setting
    target_volume: str
        The target volume to which the warp should be applied
    output_name: str
        The filename under which to save the new transformed image
    transform: str
        The filename of the warp (assumed to be a .nii.gz file)

    See also
    --------
    combine_all_transforms to see how to build a warp field
    """
    from fsl.wrappers import applywarp
    applywarp(target_volume,reference_volume, output_name, w=transform, rel=False)

Using these two functions should not be too hard now. Notice that in combine_all_transforms, setting any transform to None instead of the correct transform will skip the transform step in the total transformation. This way, you should be able to perform quality control. In particular, please ensure that:
- [ ] Applying ONLY motion correction transformation to the first volume yields the expected alignement (so it should be aligned with the \_moco volume.)
- [ ] Applying motion correction + epi -> anat should be aligned to anatomical
- [ ] Finally, applying motion correction + epi > anat + anat > standard should be aligned to the standard
Only once you're convinced these steps are working well should you proceed to standard space. **Remember the 1-10-100 rule! Always perform QC before moving on.**

To help you, we provide you below with the template to do such a thing, so that you don't have to worry too much about the nitty gritty details.
Focus on:
- The reference file to use 
- The transformations to provide (either a file or None)

<b>Notice this is performed only on a single volume. Indeed, if you are debugging you should avoid wasting time applying transformations on entire timeseries to quickly diagnose whether a step is working or failing.</b>

In [ ]:
import time
from fsl.wrappers import applywarp
ref=mni_template

# We show this one when selecting the first EPI (volume 0000)
target_epi = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_first-vol')
split_nbr = '0000'

# We will name its warp as split0000
warp_name = op.join(preproc_root, 'sub-001', 'func', 'sub-001_split' + split_nbr + '_epi_2_std_warp')

# Get the transformation matrix of this volume (this one is actually the unit matrix, 
# since this volume is the reference)


# -- Step 1: Combine the transformations, that is :
#    EPI -> Motion correction -> Coregistration to anatomical -> Normalization to standard
#    EPI -> Motion correction is given by the matrix in sub-001_task-sitrep_run-01_bold_moco.mat/MAT_{vol_nbr}, where {vol_nbr} is the volume number of the volume of interest
#    EPI -> Coreg to anatomical, this is the _warp.nii.gz file in func/ folder
#    Anatomical > Template is saved by flirt when doing the anatomical to template coregistration, in anat/ folder
func_2_anat= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_anat-space_warp.nii.gz')
epi_moco = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_moco.mat/', 'MAT_' + split_nbr)

s0 = time.time()
combine_all_transforms(ref, warp_name,  True, epi_2_moco=epi_moco, epi_2_anat_warp=func_2_anat, anat_2_standard_warp=anat_2_mni_trans)
s1 = time.time()

out_vol= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std_vol' + split_nbr)

# -- Step 2: Apply the transformation to our EPI
applywarp(target_epi,ref, out_vol, w=warp_name, rel=False)
s2 = time.time()

Above, we've timed the steps to estimate which one might be more expensive, between combining the transforms and applying them. Let's check:

In [ ]:
print('Transform combination time:', s1 - s0)
print('Apply transform time:', s2 - s1)

As you can clearly see, combining the transforms is more than 6 times slower than applying the final transform. As a consequence, we would like to do this step as rarely as we can.

#### 1.2.3.1 Optimizing a bit

Okay, so this step is slow. Can we make it faster? Well, yes!

Note that computing all these non linear fields <u>will</u> take time. We've seen above in fact that it is <u>the</u> most expensive step.
Now, applywarp has a neat option. It allows us to apply a transformation using a pre transformation matrix followed by the warp. Why is it cool?
Well, consider the following steps:

<center><img src="imgs/space_steps.png"></center>

<br>
Let's group transforms in two potential ways:

<center><img src="imgs/two_ways_grouping.png" style="max-width:1200px;"></center>
In other words, we consider <b>grouping up the transformations to apply them</b>. As you will see, computing transformations can take time when they are non linear, whereas applying them is comparatively fast. We will investigate both grouping all transforms together or grouping all transforms which follow motion correction together.

Let's compare the two methods, runtime wise:

In [ ]:
# ----- START OF METHOD 1 
# In this method, we compute the transform from start to finish and apply it
s0 = time.time()
combine_all_transforms(ref, warp_name,  True, epi_2_moco=epi_moco, epi_2_anat_warp=func_2_anat, anat_2_standard_warp=anat_2_mni_trans)
s1 = time.time()
out_vol= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std_vol' + split_nbr + '_v1')
#applywarp(target_epi,ref, out_vol, w=warp_name, rel=False)
subprocess.run(['applywarp', '-i', target_epi, '-r', ref, '-o', out_vol, '-w', warp_name, '--abs'])
s2 = time.time()
# ----- START OF METHOD 2
# In this method, we compute the transform only post motion correction. We apply the motion correction and then the warp
combine_all_transforms(ref, warp_name,  True, epi_2_moco=None, epi_2_anat_warp=func_2_anat, anat_2_standard_warp=anat_2_mni_trans)
s3 = time.time()
out_vol= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std_vol' + split_nbr + '_v2')
subprocess.run(['applywarp', '-i', target_epi, '-r', ref, '-o', out_vol, '-w', warp_name, '--abs', '--premat={}'.format(epi_moco)])
s4 = time.time()

print('Method 1 runtime:', s2 - s0, '({} for combination, {} to apply)'.format(s1 - s0, s2 - s1))
print('Method 2 runtime:', s4 - s2, '({} for combination, {} to apply)'.format(s3 - s2, s4 - s3))

To convince you that the two produced images are almost identical (you might notice differences on the order of the $10^{-3}$, but consider the relative error this entails and why such an error might happen):

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(ref)
fsleyesDisplay.load(op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std_vol' + split_nbr + '_v1'))
fsleyesDisplay.load(op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std_vol' + split_nbr + '_v2'))

Why does it matter? Well, just applying a back-of the envelope calculation, the first method takes 122s per volume, while the second method takes 87 seconds to combine **once** the transforms excluding motion correction, and 4 seconds per volume to apply the transforms including motion correction. If we plot the two with an increasing number of volumes, we can see why this quickly becomes relevant:

In [ ]:
import matplotlib.pyplot as plt

x = np.arange(0, 1000, 10)
plt.plot(x, x*122, label='Method 1')
plt.plot(x, 87 + x*4, label='Method 2')
plt.xlabel('Number of volumes')
plt.ylabel('Runtime (seconds) [LOG SCALE]')
plt.legend()
plt.yscale('log')
plt.show()

Hopefully, you're convinced that:
- We don't lose anything using method 2 imaging-wise
- We have a benefit in using method 2, computation-wise


### 1.2.4 Applying the transformation to the entire timeseries at last

With all this in mind, let's now apply our transformation to all our volumes! The steps are:

1. Split our EPI into all individual volumes (remember: applywarp only works on a single 3D image but our EPI is 4D).
2. Combine all transformations from EPI after motion correction all the way to standard space **once**. 
3. Use applywarp for every volume, passing the motion correction transform of this volume and the EPI > standard space warp
4. Combine back all volumes as a single 4D EPI in standard space

Let's make sure you understand why!

In [ ]:
interactive_MCQ(4,4)

In [ ]:
# We will split our starting EPI volume across time 
split_target = original_epi
split_name = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_split')

subprocess.run(['fslsplit', split_target, split_name, '-t'])

Let's have a look at our folder structure now.

In [ ]:
print_dir_tree(bids_root,max_depth=5)

As you can see, a lot of new volumes have appeared. These are the split, individual volumes of our EPI.

Great, let's now combine the different transforms EXCEPT motion correction, with method 2.

In [ ]:
import concurrent.futures
from tqdm import tqdm

# Let's combine the different transforms EXCEPT motion correction!
warp_name = op.join(preproc_root, 'sub-001', 'func', 'sub-001_epi_moco_2_std_warp')

print("Starting to combine transforms...")
combine_all_transforms(ref, warp_name,  True, epi_2_moco=None, epi_2_anat_warp=func_2_anat, anat_2_standard_warp=anat_2_mni_trans)
print("Done, moving on to application of transforms...")



###########
# Now apply transformation to all our volumes.
# We will remember the volumes as well, to group them back afterwards.
##########

# Notice that we are sorting the volumes here! This is important, to make sure we don't get them in random order :)
split_vols = sorted(glob.glob(op.join(preproc_root, 'sub-001', 'func', '*_bold_split*')))


# Define a function that wraps subprocess.run()
def run_subprocess(split_vol, vol_nbr):
    """
    SAFETY GOGGLES ON
    This function launches applywarp in parallel to reach complete result quicker

    Parameters
    -----------
    split_vol: str
        Path to the volume on which to apply the transformation
    vol_nbr: str
        Number of the volume in the timeserie. Useful to reorder volumes after the fact, since parallelisation does not honour order.

    Returns
    -------
    out_vol: str
        Path to the transformed volume
    vol_nbr: str
        Number of the volume in the timeserie. Useful to reorder volumes after the fact.
    """
    try:
        split_nbr = split_vol.split('_')[-1].split('.')[0].split('split')[1]
        epi_moco = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_moco.mat/', 'MAT_' + split_nbr)
        out_vol= op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std_vol' + split_nbr)
        result = subprocess.run(['applywarp', '-i', split_vol, '-r', ref, '-o', out_vol, '-w', warp_name, '--abs', '--premat={}'.format(epi_moco)], check=True)
        return out_vol, vol_nbr
    except subprocess.CalledProcessError as e:
        return f"applywarp for volume '{split_vol}' failed with error: {e.stderr.decode('utf-8')}"


produced_vols = [None]*len(split_vols)
# Initialize ThreadPoolExecutor and the progress bar
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Use tqdm to wrap the futures
    with tqdm(total=len(split_vols)) as progress:
        # Launch subprocesses in parallel
        futures = {executor.submit(run_subprocess, vol,i): vol for i,vol in enumerate(split_vols)}

        # Process completed tasks
        for future in concurrent.futures.as_completed(futures):
            out_vol, vol_nbr = future.result()  # Get the result of the subprocess
            produced_vols[vol_nbr] = out_vol
            # Update the progress bar for each completed task
            progress.update(1)

Great, now we finally have our volumes!
We can finally analyze them, as you will see next week!

### 1.2.5 Grouping the volumes together (Optional)

<b>This section is purely optional, you can readily skip it if you want.</b>

Our volumes are now individually separated, but we started with a 4D volume...
Perhaps we'd like to have them back as an individual volume, to visualize them better?
The part below aims precisely at doing that for you.

In theory we should be able to group everything back. The issue is, our fMRI is now interpolated at 1 x 1 x 1 mm3 resolution, so it won't fit in RAM on this virtual machine, where a high performance computing cluster would do this without any issue.

There are two answers to this issue. The first, used by fMRIprep, is to play it smart and actually never modify the fMRI's resolution.

The second is to use to group back the files a memory map, that is to say a file on disk from which we only read the parts we need (VERY useful when you do not have enough RAM). Because writing to disk is very slow, we do it in a batch approach below.

<div class="warning" style='background-color:#C04000; color: #112A46; border-left: solid #C04000 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b>⚠️  This part is TIME INTENSIVE and it is OPTIONAL ⚠️ </b></p>
<p style='text-indent: 10px;'>
    This part will take TIME (about an hour, in fact)
It is probably better if you jump ahead to another part of the notebook and start reading it while this is running, or just do not run it at all and remember this in case you absolutely need it later. This hour is probably better utilized somewhere else, right ? ;)</p>
</span>
</div>

In [ ]:
import nibabel as nib
import progressbar

first_vol = nib.load(produced_vols[0])
v_shape = first_vol.get_fdata().shape

preproc_root = '/home/jovyan/Data/dataset/ds004226/derivatives/preprocessed_data'
filename = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std.dat')
large_array = np.memmap(filename, dtype=np.float64, mode='w+', shape=(v_shape[0],v_shape[1],v_shape[2], len(produced_vols)))

batch_size = len(produced_vols)//4

A = np.zeros((v_shape[0],v_shape[1],v_shape[2], batch_size))

with progressbar.ProgressBar(max_value=len(produced_vols)) as bar:
    for batch_i in range(4):
        print('Starting for batch {}/4'.format(batch_i+1))
        start_batch = batch_size * batch_i
        end_batch = min(batch_size * (batch_i+1),len(produced_vols))
        max_len = end_batch - start_batch + 1
        for i in range(start_batch, end_batch):
            vol = nib.load(produced_vols[i])
            A[:,:,:,i-start_batch] = vol.get_fdata()
            bar.update(i)
        large_array[:,:,:, start_batch:end_batch] = A[:,:,:,:max_len]

In [ ]:
# Now, save to Nifti file using Nibabel

# Step 1: Ensure all changes of the memmap are flushed to disk and then close it
#large_array.flush()
#del large_array
print("Done flushing mmap")
large_array = np.memmap(filename, dtype=np.float64, mode='r', shape=(v_shape[0],v_shape[1],v_shape[2], len(produced_vols)))

# Step 2: Modify the header to indicate that we have 4D data, and specify its TR.
header = first_vol.header.copy()  # Copy the header of the first volume (to get right resolution, affine, Q-form etc)
header['dim'][0] = 4  # Specifies that this is a 4D dataset
header['dim'][1:5] = large_array.shape  # Update dimensions (x, y, z, t)
header['pixdim'][4] = 1.5  # Set the TR in the 4th dimension. You can see the TR of the data by looking at your original EPI series with fslhd, remember ;)
print("Done with header")

# Step 3: Create the Nifti1 image and save it to disk
mni_epi = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_mni.nii.gz')
img = nib.Nifti1Image(large_array, first_vol.affine, first_vol.header)
print("Done creating the image")
img.to_filename(mni_epi)
print("Done writing it to disk")

Finally, we can move on to removing all the temporary files we used, to end up with only one clean Nifti file :)

In [ ]:
os.system('rm -rf {}'.format(op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_split*.nii.gz')))

Now, let's check we did a proper job. If we did a proper mapping, we should definitely observe the EPI positioned on the anatomical in MNI space. How well did we do?

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(ref)
fsleyesDisplay.load(produced_vols[0])

## 1.3 Where are we?

<table>
    <tr><th style='text-align:justify;'>Data type</th><th style='text-align:justify;'>Step name </th><th style='text-align:justify;'>Details of the step</th><th style='text-align:justify;'>FSL tool </th></tr>
    <tr><th>Functional</th><th></th><th></th></tr>
    <tr><td></td><td style='text-align:left;'>Field unwarping</td><td style='text-align:justify;'>Correction distortions induced by inhomogeneities of the b0 field through maps acquired specifically to measure this field called fieldmaps.</td><td style='text-align:justify;'>FUGUE (but also FLIRT - see below)</td></tr>
    <tr><td></td><td style='text-align:left;'>Slice-timing correction</td><td style='text-align:justify;'>Accounting for the difference in acquisition between the slices that make up a volume to interpolate back voxels to a fixed time reference.</td><td style='text-align:justify;'>slicetimer</td></tr>
</table>

NOTE:
epi_reg is still using FLIRT under the hood! To be more specific, it is using flirt setup with its search cost as BBR. If you look through FLIRT's options, you'll notice that many more options are open to you:
<img src="imgs/flirt_options.png"/>

Feel free to explore the effect of different search costs :) But remember: not all costs are born equal when registering images **across modalities**!

## 1.4 Slice time correction

The importance of this step is still being investigated in fMRI literature. See <a href="https://www.frontiersin.org/articles/10.3389/fnins.2019.00821/full">here</a> for an in-depth analysis of its impact on the pipeline. One of the take-aways from this reference is that slice-time correction together with motion correction does improve results of fMRI analysis and does not hurt.
Doing it before or after MC is not clear, as you can see in the reference above, so we're *choosing* here to showcase it after motion correction, but only time and further investigations will tell if there's a good order :)


#### 1.4.1 A toy example

To help you understand the underlying theory of slice-time correction, we will start from a rather unreasonable case on synthetic data, which will help you better visualize how slice-time correction affects the patterns.

As you'll see, this step can only be performed if you have knowledge of the way in which slices were acquired. Most of the time, fortunately, this is easy to recover. If it is not present in your data, you can ask the scanner operator to find out which sequence was used for your data acquisition.

Let's get started!

In [ ]:
%run generate_smileys.py

First load the file named "ground_truth_modulation.nii" in FSLeyes to visualize it. The data is 4D, go ahead a play a bit around to see exactly what happens during the sequence! (don't forget in case of flickering to untick the "Synchronize movie update" option of FSLeyes).

What you see is the ground truth, ie: the real phenomenon as it plays out!

We now have an acquisition sequence. It performs in slices, but not in a linear order.
Our sequence is even weirder: at a given time, not one but 9 slices are made at the same time! 

In [ ]:
from scipy.interpolate import InterpolatedUnivariateSpline as Interp
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

def save_array_asnib(array, save_name):
    """
    Save a numpy array as a Nifti file.
    The nifti is considered to have identity transformation matrix and to be unsigned int.
    If you wish to use this function for float data, you should remove the .astype(np.uint8) ;)

    Parameters
    ----------
    array: np.ndarray
        The array to save
    save_name: str
        Path to which the array will be saved
    """
    img = nib.Nifti1Image(array.astype(np.uint8), np.eye(4))
    nib.save(img, save_name)
    
def check_dims(axis_len, seq_len):
    """
    Checks that axis length and seq length are equal. Otherwise, raise an Exception.

    Parameters
    ----------
    axis_len: int
        Length of the axis (number of elements along the axis of the array in your case)
    seq_len: int
        Length of the sequence
    """
    if axis_len != seq_len:
        raise Exception('The number of slices in the sequence is different from the number of slices available in the axis. Are you sure this is the right axis?')

def reslice_with_timings(slice_dir, slice_sequence,input_data, original_times):
    """
    Perform slice timing correction according to slice sequence timing and slice direction.

    Parameters
    ----------

    slice_dir: char (can be 'x', 'y', or 'z')
        Slice direction, the encoding direction of the sequence.
    slice_sequence: np.ndarray
        Matrix representing which slices are acquired in which order.
    input_data: np.ndarray
        The matrix that was acquired by the slice sequence and which we wish to correct
    original_times: np.ndarray
        The times at which the images are acquired and where we would like to reinterpolate back our slices.

    Returns
    --------
    output_data: np.ndarray
        The resliced data matrix, result of the slice-timing correction.
    """
    assert(original_times.size == input_data.shape[3])
    
    n_acqs_per_tr = slice_seq.shape[0]
    n_multibands = slice_seq.shape[1]
    n_slices = slice_seq.size
    
    r= -1
    if slice_dir=='x':
        # For each slice in x, interpolate!
        n = input_data.shape[0]
        check_dims(n, n_slices)
        r=0
    elif slice_dir == 'y':
        # For each slice in y, interpolate!
        n = input_data.shape[1]
        check_dims(n, n_slices)
        r=1
    elif slice_dir == 'z':
        # For each slice in z, interpolate!
        n = input_data.shape[2]
        check_dims(n, n_slices)
        r=2    
    else:
        # Undefined yo
        raise Exception('Invalid dimension! Should be x, y or z')
    # Reshape the input data to have r as first dimension!
    input_data = np.swapaxes(input_data, 0, r)
    
    output_data = np.zeros(input_data.shape)
    print(output_data.shape)

    y_s = input_data.shape[1]
    z_s = input_data.shape[2]
    
    # Now, on the first axis, we will iterate over slices :)
    for b in range(0, n_acqs_per_tr):
        time_slice = original_times + b*1./n_acqs_per_tr
        # For all slices acquired together in the multiband
        slices = slice_seq[b]
        print('---------')
        print(time_slice)
        print(slices)
        for s in range(0, n_multibands):
            sl = slices[s]
            for y in range(0, y_s):
                for z in range(0, z_s):
                    lin_interper = Interp(time_slice, input_data[sl, y, z, :], k=1)
                    output_data[sl, y, z, :] = lin_interper(original_times)
    # Now that this very inefficient method is done, we should remember to swap back the axis :)
    input_data =np.swapaxes(input_data, r, 0)
    output_data=np.swapaxes(output_data, r, 0)

    output_data[output_data < 0] = 0
    return output_data

In [ ]:
slice_seq = np.arange(0, 99).reshape((11, - 1))
slice_seq

As you can see, these slices are acquired in a sequential order that can be called either ascending or descending depending on your convention!
It means that the slices are obtained successively.

There are different types of slicing, which depends on your sequence. Notice that in our example we acquire several slices simultaneously (for example, slices 0 up to 9 are all acquired together!). This could be some multiband acquisition, for instance, but really it is mostly to help you visualize the effect of slice timing for the exercise. In practice the slices are defined to sample the signal in the most appropriate way, so our toy sequence will likely be too crude :)

In any case, we had some ground truth signal, that you can visualize in ground_truth_modulation.nii (don't forget to play the movie with the box unticked!)

This signal represents the true signal that we want to acquire.
The participant steps in an MRI, and the scanner operator uses the sequence we've defined above:

```
array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
       [ 9, 10, 11, 12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23, 24, 25, 26],
       [27, 28, 29, 30, 31, 32, 33, 34, 35],
       [36, 37, 38, 39, 40, 41, 42, 43, 44],
       [45, 46, 47, 48, 49, 50, 51, 52, 53],
       [54, 55, 56, 57, 58, 59, 60, 61, 62],
       [63, 64, 65, 66, 67, 68, 69, 70, 71],
       [72, 73, 74, 75, 76, 77, 78, 79, 80],
       [81, 82, 83, 84, 85, 86, 87, 88, 89],
       [90, 91, 92, 93, 94, 95, 96, 97, 98]])
```

What this really means is that we acquire 9 slices at the same time and then move on to the next slices to acquire, we acquire 9 of them and so on.

Your task will be two-folds:
1. Understand which axis the sequence was applied on, ie along which direction (X, Y or Z) was slicing performed
2. With this simple knowledge, apply a slice-timing correction algorithm 

This cute smiley will represent our neurological data! Every time point, the behaviour is simple: the smiley's intensity increases across time!
Here is what it looks like in FSLeyes:

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load('ground_truth_modulation.nii')
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[0]).cmap = 'hot'

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load('ground_truth_upsampled_modulation.nii')
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[0]).cmap = 'hot'

Using the following slice sequence, we have acquired our smiley signal.
The resulting timeseries is represented in the acquired_modulation file. 

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load('acquired_modulation.nii')
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[0]).cmap = 'hot'

Oh no! What went wrong?

Well, if you think about it, nothing. It is simply that acquiring every slice takes time. During this time, the signal is evolving, so we're a little late in our acquisition, which causes the drift you're seeing.

This is what you observe in the acquired modulation file. The slice that is at the top - which is the last slice - also turns out to be the one with highest value. At timestep 0, it is in fact almost equal to 10 - the value of the ground truth at timestep 1 !

### 1.4.2 Correcting the delay

The heart of slice-timing correction is an interpolation in time.
Because the timing of the slices is wrong, we account for it by interpolating back to some reference time. This is how we obtain the resliced data.
For this, we need some informations. The first is the sequence in which slices are acquired, to know the lag of each slice. We also need the axis along which slices are acquired.

Your task, in this simplified example, based **only on the abnormal smiley visualization in FSLeyes** and the knowledge of the sequence (that is: a bottom-up sequence with 9 simultaneous bands in every slice), is to determine the direction of the phase encoding, ie:

In [ ]:
interactive_MCQ(4,5)

Based on your answer, in the following cell, fill in the phase_encode variable, with either 'x', 'y' or 'z' (with the quotation marks!).

In [ ]:
phase_encode = 'y'

We will now conduct slice-timing correction: the idea is simply to interpolate back the slices in time along the slice direction. Easy right? Let's do it:

In [ ]:
smile_ts = nib.load('ground_truth_modulation.nii').get_fdata()
smile_resampled = nib.load('acquired_modulation.nii').get_fdata()
resliced_data = reslice_with_timings(phase_encode, slice_seq, smile_resampled, np.arange(0,9))
save_array_asnib(resliced_data.astype(np.uint8), 'resliced_data.nii')

fsleyesDisplay.resetOverlays()
fsleyesDisplay.load('resliced_data.nii')
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[0]).cmap = 'hot'

Are you convinced on this toy example we did a not-too bad job?

### 1.4.3  Application to real data

We have shown you the basic principle, but the application to real data requires some specific informations.
You need the following ingredients:
- When was each slice acquired in the sequence: **(Slice timing)**
- Along which axis were the slices acquired: **Phase direction**
- How much time we take to acquire all slices: **TR**

Let's go back to our practical dataset to extract these informations. Can you find them, when looking through the JSON sidecar?

In [ ]:
data = get_json_from_file(op.join(bids_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold.json'))
data

This data is actually a dictionary. We can thus extract the slice timing as an array directly from it. For example, to extract TaskName, we would use:
```python
data['TaskName']
```

Go ahead and extract the slice timing array, and store it in the slice_timing variable.

In [ ]:
slice_timing = data['SliceTiming'] # Replace with the appropriate key (have a look above!)

Now, we might want to know where our slices are, ie along which axis, right? Typically it is along the z-direction, but we're better off if we check! Using FSLeyes, determine how many slices each axis has **for the functional data of interest**. You should thus open the relevant functional file in FSLeyes to answer this question.


<div class="warning" style='background-color:#90EE90; color: #112A46; border-left: solid #805AD5 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b>Using FSL command line</b></p>
<p style='text-indent: 10px;'>To figure out the dimensions of an MRI image, a faster option - if you have FSL installed directly - is to run the command line command:
    <blockquote>fslhd [your_volume]</blockquote>
This will give you all informations contained within the header of the NIfti file. For example, running the command for our volume will easily allow us to access the slice informations:
    <img src="imgs/fslhd_capture.png"></p>
</span>
</div>
Let's compare now with the amount of slices we have in our acquisition. We can consider simply the number of timings for this

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(op.join(bids_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold.nii.gz'))

In [ ]:
len(slice_timing)

So we have 52 slices in our slice timings, and you likely found 77 slices on the X axis, 77 axis on the Y axis and 51 slices on the Z axis. As a consequence, Z is the axis where the slices were acquired!
Great, so we know which axis we want, we know the slice timings, but we still need to know the TR. This information is also in the JSON sidecar! Extract it now!

In [ ]:
tr = data['RepetitionTime'] # Extract the TR from the sidecar's appropriate field
tr

To now perform the correction, we need to apply FSL's slicetimer command. For this, we need to save the timings first to their own separate file! Instead of giving the slice timings, we will provide instead the slice **order** (ie which slice was done in which order) and let FSL figure out how to best correct based on this information.

Let's do it.

In [ ]:
slice_order = np.argsort(slice_timing) + 1

# Write to a file the corresponding sorted timings :)
timing_path = op.join(preproc_root,  'sub-001', 'func', 'sub-001_task-sitrep_run-01_slice-timings.txt')
file = open(timing_path, mode='w')
for t in slice_order:
    file.write(str(t) + '\n')
file.close()

Finally we can call slicetimer from a terminal!

In [ ]:
file_to_realign = op.join(bids_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold')
output_target = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_slice-corr')

subprocess.run(['slicetimer', '-i', file_to_realign, '-o', output_target, '-r', str(tr), '-d', str(3), '--ocustom={}'.format(timing_path)])
#cmd = 'slicetimer -i ' + file_to_realign + ' -o ' + output_target + ' -r ' + str(tr) + ' -d 3 --ocustom=' + timing_path
#os.system(cmd)

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(file_to_realign)
fsleyesDisplay.load(output_target)

As you would notice, visual differences due to slice-timing correction are not too obvious in general.

There are unfortunately cases where it can go very wrong, due to the interpolation nature of the approach.
Consider the four images below.

<div class="fit">
<img src="imgs/slice_uncorr.png" max-width="1400px;">
</div>


Had we launched it on the unscrubbed data, we would really notice the impact on the first volume. <span style="color:red;">Notice that you should in general **never** do this, as you would introduce lots of noise and garbage in your data.</span> Prefer to first clean all that is weird and then perform steps that might not bring a visible improvement rather than starting with data that is so bad that you can visually see changes when running above steps.
 <div class="row">
    <img src="imgs/slice_uncorr.png" style="width:100%">
      <center>First volume without slice correction</center>
    <img src="imgs/slice_corr.png" style="width:100%">
       <center>First volume with slice correction: the staircase has been more or less mitigated but the result is still imperfect...</center>
    <center>**And because of the linear interpolation, the garbage of volume 0 was spilled to volume 1!!!**</center>
    <img src="imgs/vol1_slice_uncorr.png" style="width:100%">
      <center>Second volume without slice correction</center>
    <img src="imgs/vol1_slice_corr.png" style="width:100%">
       <center>Second volume with slice correction: the result is worse than before...</center>
</div> 


The message here is: 
- **always** perform QC between your steps
- no algorithm can turn trash to gold. Remove the faulty volumes or you'll likely have a garbage-in garbage-out scenario. 

<u>Remember the 1 - 10 -100 dollar rule! It is much easier to avoid errors than compensate for them.</u>

# MRI and fMRI preprocessing: conclusion

You have reached the end the preprocessing for MRI and fMRI. 

As you can see, there are many fairly involved steps that need to be conducted.

To normalize choices and results, software solutions exist, such as the excellent <a href="https://fmriprep.org/en/stable/">fmriprep</a>, to conduct automatically all steps for you while providing you with quality checks to verify that all went well, as no algorithm can replace your expert eye to inspect potential artefacts and remove them.

These tools are nonetheless precious to help different groups follow same systematic choices of preprocessing, both in parameters and order of application for each method, leading to more reproducible science in the long run. (But they require a big RAM and take hours to run, which is why we've avoided them for the purpose of this tutorial :) )

Let's wrap up what you have learnt.


<table>
    <tr><th style='text-align:justify;'>Data type</th><th style='text-align:justify;'>Step name </th><th style='text-align:justify;'>Details of the step</th><th style='text-align:justify;'>FSL tool </th></tr>
    <tr><th>Anatomical</th><td></td><td></td></tr>
    <tr><td></td><td style='text-align:justify;'>Skull stripping</td><td style='text-align:justify;'>Removing skull and surrounding tissues to keep only the brain</td><td style='text-align:justify;'>BET</td></tr>
    <tr><td></td><td style='text-align:justify;'>Segmentation</td><td style='text-align:justify;'>Segmenting brain tissues based on their contrasts</td><td style='text-align:justify;'>FAST</td></tr>
    <tr><td></td><td style='text-align:justify;'>Normalization</td><td style='text-align:justify;'>Mapping participant's brain to a reference brain, making its orientation and scale match so that comparison across participants become feasible.</td><td style='text-align:justify;'>FLIRT</td></tr>
    <tr><th>Functional</th><th></th><th></th></tr>
    <tr><td></td><td style='text-align:left;'>First few volumes removal</td><td style='text-align:justify;'>Removing volumes for which the B0 field is still not stable and that could contaminate all our data if left unchecked.</td><td style='text-align:justify;'>fslroi</td></tr>
    <tr><td></td><td style='text-align:left;'>Motion correction</td><td style='text-align:justify;'>Realignment of fMRI volumes to a common reference - typically one volume or the average of the volumes - to correct for inter-volume motion. The extracted motion parameters can be used for subsequent analysis (see GLM in one week!)</td><td style='text-align:justify;'>MCFLIRT (which is one suboption of FLIRT in fact)</td></tr>
    <tr><td></td><td style='text-align:left;'>Fieldmap preparation</td><td style='text-align:justify;'>Field maps can be used to create a distortion field to correct...Distortions. </td><td style='text-align:justify;'>topup, FUGUE</td></tr>
    <tr><td></td><td style='text-align:left;'>Coregistration</td><td style='text-align:justify;'>Realignment of fMRI volumes to anatomical space - the subject's own MRI is typically used. We can include susceptibility-distortion correction with fieldmaps. We compute this transformation only for the volume we used as reference in MCFLIRT. Then, we apply it to all other volumes in the EPI.</td><td style='text-align:justify;'>epi_reg (FLIRT with boundary-based registration)</td></tr>
    <tr><td></td><td style='text-align:left;'>Normalization</td><td style='text-align:justify;'>Putting the EPI in a template space, such as MNI. This is done by applying the transformation of anatomical space to MNI space, which was computed in the anatomical preprocessing. Note that we typically like to combine this transform with coregistration to do everything in one go.</td><td style='text-align:justify;'>applywarp to apply combined warps, otherwise (if going transform by transform), FLIRT with applyxfm option</td></tr>
</table>


If you have any more questions, both on these tools and on preprocessing, do not hesitate!

<p style="color:red;font-size:20px;">To release all fsleyes resources and to have interactivity below, you will now need to STOP the kernel. Start running the cells from Part 2 onwards after that.</p>

# Part 2: Get acquainted with fNIRS data preprocessing

In contrast to functional magnetic resonance imaging (fMRI), functional Near-Infrared Spectroscopy (fNIRS) distinguishes itself with portability and adaptability, making it especially suitable for research involving intricate populations like infants, tasks characterized by motion, and real-world settings.

Nevertheless, it is important to acknowledge that collecting fNIRS data requires rigorous preprocessing. This requirement arises from fNIRS's susceptibility to 1) superficial physiological interferences, such as those stemming from scalp blood flow and 2) motion artifacts, especially those resulting from sensor displacement.

In [5]:
import os.path as op
from mne.preprocessing.nirs import temporal_derivative_distribution_repair

## 2.1 Data set

### 2.1.1 Description

We will work on a set of hemodynamic data measured on one subject during a finger tapping paradigm with three conditions: 1) Tapping the left thumb to fingers, 2) Tapping the right thumb to fingers and 3) A control when nothing happens. Each tapping lasts 5 seconds and there are 30 trials in each condition.The recording was performed using fNIRS sensors located over motor areas of the cortex. 

Data were provided by Luke, R., & McAlpine, D. (2021). fNIRS Finger Tapping Data in BIDS Format (Version v0.0.1) (https://doi.org/10.5281/zenodo.5529797),

### 2.1.2 Loading

Load the dataset by running the next cell.

In [1]:
import mne
import os.path as op
#Download dataset
fnirs_data_folder = mne.datasets.fnirs_motor.data_path(path='/home/jovyan/Data/mne_data/')
#Get the path for the dataset folder 
fnirs_data_folder=op.join(fnirs_data_folder, 'Participant-1')
#Load the dataset 
raw_intensity = mne.io.read_raw_nirx(fnirs_data_folder, verbose=True, preload=True)

Loading /home/jovyan/Data/mne_data/MNE-fNIRS-motor-data/Participant-1
Reading 0 ... 23238  =      0.000 ...  2974.464 secs...


### 2.1.3 Recording setting

Display and read information on the recording setting such as the number of channels, the file duration and the sampling frequency by runing the next cell. 

In [8]:
#Display recording setting
raw_intensity

<RawNIRX | Participant-1, 56 x 23239 (2974.5 s), ~10.0 MB, data loaded>

In [ ]:
interactive_MCQ(4,6)

The placement of fNIRS sensors holds significance for achieving **a good spatial resolution** and an **accurate sensor placement**.

Let's take a look at the locations of sensors.

In [9]:
!pip install "pyvista[jupyter]"
!pip install pyvistaqt
!pip install pyqt5
!pip install qtconsole pyqt5

In [2]:
%matplotlib qt5
subjects_dir = op.join(mne.datasets.sample.data_path('/home/jovyan/Data/mne_data'), "subjects")

brain = mne.viz.Brain(
subject="fsaverage", subjects_dir=subjects_dir, background="black", cortex="0.5"
)
brain.add_sensors(
    raw_intensity.info,
    trans="fsaverage",
    fnirs=["channels", "pairs", "sources", "detectors"],
)
brain.show_view(azimuth=20, elevation=60, distance=400)

QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-jovyan'


Using pyvistaqt 3d backend.
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
Channel types::	fnirs_cw_amplitude: 56


<p style="color:green"> See below the expected 3D visualization of the fNIRS sensors placed over the head: </p>

<img src="imgs/3Dmap.png">

In the 3D visualization, we represent source-detector pairs or channels as lines. 

<span style="color:blue; text-decoration:underline;">Multiple Choice Question</span>

*What is the cortical region covered by this measurement ?*
1. Motor cortex
2. Visual cortex

   <p style="color:green"> In this finger tapping task, sensors are placed over the motor cortex. This region is located in the dorsal portion of the frontal lobe. </p>


### 2.1.4 Experimental design

Let's now have a look at the experimental design used.

The experimental designs most commonly employed by auditory fNIRS researchers are block- and event-related designs. In an event related design, each task is presented individually for a short amount of time e.g., 3 seconds. In this way, tasks can be more  randomized,  rather  than  being  blocked  together  by  condition. Conversely, in a block-related design, blocks of tasks, each lasting at least 10 seconds, recur before intervals of rest. 

The choice of the experimental design depends on a range of factors, including the statistical power, the duration of the experiment, and whether the design provides the flexibility to study the effect of interest. While the block design might lead to higher detection power, it can also induce learning and boredom effects which may bias the results. On the other hand, event-related designs reduce the effects of learning, boredom while exhibiting loss in detection power. 

<span style="color:blue; text-decoration:underline;">Open question</span>

*What are the analytical challenges associated with employing a block-related experimental design featuring continuous stimulation and brief rest blocks?*

   <p style="color:green"> In a block-related experimental design with continuous stimulation and brief rest blocks, we can end-up having a non-linear summation of the hemodynamic responses to stimuli. This non-linearity can cause a mismatch between the predicted  summation of responses obtained with linear approaches (e.g., averaging or general linear modelling) and the actual observed data. </p>







Run the next cell to display the sequence of events used in this experiment.

In [ ]:
#Load the event annotations
events, event_dict=mne.events_from_annotations(raw_intensity,verbose=False)
#Assign each label to the event (based on recording setting)
event_dict={'Control':1,'Tapping/Left':4,'Tapping/Right':3,'ExperimentEnds':2}
#Display the sequence of events 
plt.rcParams["figure.figsize"]=(10,6)
mne.viz.plot_events(events,event_id=event_dict,sfreq=raw_intensity.info['sfreq']);

As you can see, there is a significant gap or interstimulus interval between each individual stimulus. We can thus conclude that an event-related design was employed in this experiment.

Now that we have gained an understanding of the experiment and recording processes, we can start the visualization of the raw data collected. To proceed, run the next cell.

In [ ]:
#Create a figure window
plt.rcParams["figure.figsize"]=(40,40)
#Plot time-series of raw light intensity data 
mne.viz.plot_raw(raw_intensity,start=120,duration=80,n_channels=56,show_scrollbars=False)

In this plot, we can observe the time-series of the data collected, represented as the change in light intensity resulting from light absorption at specific wavelengths. Oxygenated hemoglobin (HbO) absorbs light at "850"nm, while deoxygenated hemoglobin (HbR) absorbs light at "760"nm. Each channel comprises a pair consisting of a source (labeled as "S" in the plot) and a detector (labeled as "D" in the plot). 

Here, we are showing all the channels (y-axis) but, for clarity, we are only focusing on a specific timeframe, spanning from 120 to 200 seconds (x-axis). If you wish to customize the visualization parameters, please refer to the documentation of [mne.viz.plot_raw](https://mne.tools/stable/generated/mne.viz.plot_raw.html).

<span style="color:blue; text-decoration:underline;">Open question</span>

*Visually, are you able to localize a motion artifact?*

   <p style="color:green"> There are two types of motion artefacts we can visualize in this graph: a spike between 185-190s and a baseline shift in the S8_D8 850 channel. These two types of artefact can easily happen in motor tasks. </p>

## 2.2 Preprocess the data

A typical preprocessing pipeline for fNIRS data includes the following key steps:

<hr>

*Step 1: Raw Intensity to Optical Density Conversion*

<hr> 

*Step 2: Motion Artifact Removal*

<p style='margin-top:1em;'>💡 This step is more effective when performed at the beginning of the pipeline to prevent the propagation of errors across wavelengths and minimizes cross talk between signals obtained at different wavelengths.

<hr> 

*Step 3: Optical Density to Concentration Conversion*

<hr>

*Step 4: Physiological Oscillation Filtering*

<p style='margin-top:1em;'>💡 This step is more effective when performed after the conversion step using the modified Beer-Lambert law, as physiological sources of error impact HbO and HbR in a manner consistent with the Beer-Lambert equation.
    
    
<hr>

### 2.2.1 Converting raw intensity data to optical density

The first step is to convert the changea in light intensity into changes in optical density using the modified Beer-Lambert law (Eq 1). 

\begin{equation}
OD_\lambda - OD_{R\lambda} = log\frac{I_o}{I} (1)
\end{equation}

With:
- $OD_\lambda$ the optical density of the medium for a given wavelength $\lambda$
- $OD_{R\lambda}$ the optical density of light scattering within human tissue
- $I_o$ the incident light intensity 
- $I$ the transmitted light intensity 

Run the next cell to convert raw intensity into optical density.

In [ ]:
#Convert raw intensity into optical density data
raw_od=mne.preprocessing.nirs.optical_density(raw_intensity)

### 2.2.2 Correcting motion artefacts

In fNIRS data, two types of artifacts are commonly encountered: baseline shifts (indicating sensor displacement without returning to the initial position) and spike artifacts (characterized by sensor oscillations). As you correctly guessed earlier, the abrupt shift occurring around ~190 seconds in the time-series plot above corresponds to a spike artifact.

Here, we will employ a technique known as Temporal Derivative Distribution Repair (TDDR), which is designed to address motion artefacts. TDDR uses the temporal derivative of the signal to correct the signal. 

For a more comprehensive understanding of the TDDR method and its application, you can refer to the following paper:
- Frank A. Fishburn, Ruth S. Ludlum, Chandan J. Vaidya, and Andrei V. Medvedev. "Temporal Derivative Distribution Repair (TDDR): A Motion Correction Method for fNIRS." NeuroImage, 184:171–179, 2019. doi:10.1016/j.neuroimage.2018.09.025.

Run the cell below to apply the TDDR on the optical density data. 

In [ ]:
#Apply TDDR on optical density data
raw_od_preproc=temporal_derivative_distribution_repair(raw_od)

Execute the following cell to visualize the optical density data after applying TDDR. 

In [ ]:
#Create a figure window
plt.rcParams["figure.figsize"]=(40,40)
#Plot time-series of optical density data following TDDR application
mne.viz.plot_raw(raw_od_preproc,show_scrollbars=False,start=120,n_channels=56,duration=80)

<span style="color:blue; text-decoration:underline;">Open question</span>

*From a visual standpoint, can we conclude that TDDR successfully eliminated the motion artifacts from your data?*

 <p style="color:green"> While the TDDR corrected the baseline shift seen in some of the channels (e.g., S8_D8 850) it did not remove the spikes from the signal. This low sensitivity to motion-related spikes is actually one of the limits of the TDDR method. More robust methods that are however not proposed in mne-nirs library are the wavelet filtering or the spline interpolation.                                                                                                               For a comparison of motion artefact correction techniques, you can refer to the following paper:
    Cooper, R. J., Selb, J., Gagnon, L., Phillip, D., Schytz, H. W., Iversen, H. K., ... & Boas, D. A. (2012). A systematic comparison of motion artifact correction techniques for functional near-infrared spectroscopy. Frontiers in neuroscience, 6, 147.</p>



### 2.2.3 Converting optical density to concentration 

The changes in optical density are then converted into changes of concentration using the modified beer lambert law (Eq 1): 

\begin{equation}
\Delta c = \frac{\Delta OD_\lambda }{\epsilon_\lambda lB} 
\end{equation}

Whith:
- $\Delta$ c the molar concentration change (in M)
- $\epsilon_\lambda$ the moral absorption coefficient (in $M^-1 cm^-1$) for a given wavelength $\lambda$
- l the the optical pathlength 
- B the pathlength correction factor

For more information on the equation, see the reference: Delpy DT, Cope M, Zee P van der, Arridge S, Wray S, Wyatt J. Estimation of optical pathlength through tissue from direct time of flight measurements. Phys Med Biol 1988; 33: 1433-1442.

Run the cell below to compute the concentration changes.

In [ ]:
#Import required beer-lambert law related function
from mne.preprocessing.nirs import beer_lambert_law
#Convert optical density into concentration change (hemodnyamic data)
raw_haemo=beer_lambert_law(raw_od_preproc)

### 2.2.4 Filtering out physiological oscillations

Let's now filter out physiological oscillations.

To begin, we'll assess the extent of physiological oscillations within the data using power spectral density (PSD) analysis. Based on the sequence of events outlined previously, we note that stimuli for a same condition (e.g., tapping right) are presented at a rate of approximately once every 1/100 seconds. Consequently, we should anticipate a hemodynamic response to this specific stimulus occurring at this particular frequency. Any additional components identified in the PSD analysis can be attributed to other forms of oscillations, including physiological ones.

Run the following cell to initiate the signal decomposition process.

In [ ]:
#Compute the PSD of non-filtered hemodynamic data 
fig = spectrum = raw_haemo.compute_psd().plot(average = True)
fig.suptitle('Before filtering', weight='bold', size='x-large')
fig.subplots_adjust(top=0.88)
plt.show()

The PSD shows a very low peak which may correspond to the hemodynamic responses to task stimulations. On the other hand, the presence of a frequency peak at 1.25 Hz aligns with the heart rate. 

Based on the information provided, apply a low-pass filter to retain only the hemodynamic responses. You may need to utilize the filter method available in the mne.io.Raw class as outlined in the MNE documentation (https://mne.tools/stable/generated/mne.io.Raw.html).

<p style="color:green"> MNE library provides two methods for filtering raw data: filter_data() function or the method filter. Here, we will utilize the filter() method, as it is more user-friendly and integrates seamlessly with subsequent analysis steps. 

<p style="color:green"> In the context of event-related designs, the goal is to extract hemodynamic responses to specific events. In our case, we aim to compare the hemodynamic responses to different events. 
Therefore, we have three distinct signals of interest: hemodynamic responses to 1) tapping left, 2) tapping right, and 3) control events. Maintaining the stimulation frequency in these signals is crucial. To determine this stimulation frequency, 
refer to the event sequence above. You'll notice there is around 1 stimulation per 100 seconds for each event. 
In the power spectrum above, you can identify a peak at approximately 0.01Hz, likely associated to this event-related hemodynamic response. You may also want to get rid of physiological oscillations: among the most important, there is the heart rate at ~1Hz and the breathing rate at ~0.3Hz. You can see the corresponding peaks in the PSD above. Based on this, the upper pass-band edge of your low pass filter can be set to 0.2Hz. You can use transbandwidth to make a smooth transition between the passband (0.2Hz) and the stopband (0.3Hz, frequency above which you will get your signal contaminated by physiological oscillations). Filters with narrow transition bandwidths are often preferred in applications where closely spaced frequency components need to be separated such as ours. However, be aware that you cannot narrow too much the transition bandwidth with low-order filters as the ones proposed by MNE library.</p>

In [ ]:
###########
# Now apply the filter method to retain only the hemodynamic responses from your signal
##########

#And now, let's try to filter the hemodynamic data using the filter method for raw objects 
raw_haemofiltered=raw_haemo.filter(0, 0.2, h_trans_bandwidth=0.1,
                             l_trans_bandwidth=0)


<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b>💡 Pay attention! 💡</b></p>
<p style='text-indent: 10px;'> Whenever you apply a filter, you should check that you are not removing the signal of interest. For that, make sure the task stimulation frequency is not within the frequency range of your filter ! </p>
</span>
</div>

Run the next cell to plot the PSD of your filtered data.

In [ ]:
#Plot the spectrum of filtered hemodynamic data 
fig = spectrum = raw_haemofiltered.compute_psd().plot(average = True)
fig.suptitle('After filtering', weight='bold', size='x-large',y=1.1)
fig.subplots_adjust(top=0.88)
plt.show()

Run the next cell to visualize the filtered data.

In [ ]:
#Plot time-series of hemodynamic data obtained with one channel 
raw_haemofiltered.plot(start=0,duration=200,n_channels=1,show_scrollbars=False)

Congratulations ! You now have all the basics to understand and preprocess fNIRS data!  

<div class="alert alert-success">
<p><b>🎉 You've reached the end of this week's notebook! Congratulations! 🎉 </b></p>
</div>